In [57]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
#init_notebook_mode(connected=True)

In [58]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [59]:
import numpy as np
import sympy as sy
import math
import cmath

Define the function to analyze.  This function should be defined in sympy.functions and depend on one variable, $x$.

In [60]:
x = sy.Symbol('x')
def get_fx():
    f = sy.functions.exp(x)
    
    return f

Define the complex version of the function above.  This function should be defined in cmath.

In [61]:
def eval_cfx(z):
    f = cmath.exp(z)
    
    return f

Define the analysis bounds (lower and upper) and resolution (step).

In [62]:
lower, upper, step = (0, 5, 0.1)

For intended functionality, you shouldn't need to change anything below here.
Just interact with the sliders.

In [63]:
def get_fx_eval(fx, arg):    
    
    return fx.evalf(subs={x:arg})

In [64]:
def get_fx_vals(fx, name, lower, upper, step):
    x_vals = np.arange(lower, upper, step)
    y_vals = np.empty(x_vals.shape)
    for idx, x_val in np.ndenumerate(x_vals):
        y_vals[idx] = get_fx_eval(fx, x_val)
        
    return go.Scatter(name=name, x=x_vals, y=y_vals)

In [65]:
def plot_taylor(fx, x0, n):
    p = 0
    for i in range(n+1):
        p += (fx.diff(x, i).subs(x, x0)) / (math.factorial(i)) * (x - x0) ** i
    
    fx_vals = get_fx_vals(fx, str(fx), lower, upper, step)
    taylor_vals = get_fx_vals(p, 'taylor expans.', lower, upper, step)
    cur_pt = go.Scatter(x=np.array([x0]), y=np.array([get_fx_eval(fx, x0)], dtype=float),
                        text=['x0'], mode='markers+text', textposition='top center', name='tracer')
    iplot([fx_vals, taylor_vals, cur_pt])
    
    return p

In [66]:
w = interact(plot_taylor, fx=fixed(get_fx()), x0=(0,5,1), n=(0,10,1))

A Jupyter Widget

In [67]:
def get_cfx_data(x_vals, h):
    y_vals = np.empty(x_vals.shape)
    for idx, x_val in np.ndenumerate(x_vals):
        c_val = x_val + 1j*h
        y_vals[idx] = eval_cfx(c_val).imag / h
        
    return y_vals

In [68]:
def get_central_diff_data(x_vals, h):
    y_vals = np.empty(x_vals.shape)
    for idx, x_val in np.ndenumerate(x_vals):
        y_vals[idx] = (eval_cfx(x_val + h).real - eval_cfx(x_val - h).real) / (2 * h)
        
    return y_vals

In [69]:
def plot_derivs(fx, n):  
    h = 10 ** n
    x_vals = np.arange(lower, upper, step)
    
    sym_deriv = fx.diff(x, 1)
    der_y_vals = get_fx_vals(sym_deriv, str(sym_deriv), lower, upper, step)
    cfx_y_vals = get_cfx_data(x_vals, h)
    cdiff_y_vals = get_central_diff_data(x_vals, h)
    
    cfx_rel_err = (der_y_vals['y'] - cfx_y_vals) / der_y_vals['y'] * 100
    cdiff_rel_err = (der_y_vals['y'] - cdiff_y_vals) / der_y_vals['y'] * 100
    
    cfx_vals_trace = go.Scatter(name='ctse diff.', x=x_vals, y=cfx_y_vals)
    cdiff_vals_trace = go.Scatter(name='central diff.', x=x_vals, y=cdiff_y_vals)
    cfx_err_trace = go.Scatter(name='ctse diff. err.', x=x_vals, y=cfx_rel_err, yaxis='y2')
    cdiff_err_trace = go.Scatter(name='central diff. err.', x=x_vals, y=cdiff_rel_err, yaxis='y2')

    layout = go.Layout(yaxis=dict(title='Derivative Values'),
                       yaxis2=dict(title='Relative Error [%]', overlaying='y', side='right'))
    plot_data = [der_y_vals, cfx_vals_trace, cdiff_vals_trace, cfx_err_trace, cdiff_err_trace]
    comp_fig = go.Figure(data=plot_data, layout=layout)
    iplot(comp_fig)
    
    return None

Note, the step size used by the numerical differentiation formulas, $h$, is defined here as:

$$h = 10^n$$

In [70]:
cw = interact(plot_derivs, fx=fixed(get_fx()), n=(-20, 0, 1))

A Jupyter Widget